In [8]:
!wget https://drive.google.com/file/d/1Ev4hr19oF06OEink2Km6zuU9owZxHXse/view?usp=share_link

--2022-12-06 17:29:43--  https://drive.google.com/file/d/1Ev4hr19oF06OEink2Km6zuU9owZxHXse/view?usp=share_link
Resolving drive.google.com (drive.google.com)... 142.251.162.102, 142.251.162.101, 142.251.162.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.162.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=share_link’

view?usp=share_link     [ <=>                ]  72.79K  --.-KB/s    in 0.001s  

2022-12-06 17:29:43 (91.0 MB/s) - ‘view?usp=share_link’ saved [74542]



In [9]:
!7z e /content/Orders.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 20581255 bytes (20 MiB)

Extracting archive: /content/Orders.7z
--
Path = /content/Orders.7z
Type = 7z
Physical Size = 20581255
Headers Size = 130
Method = LZMA2:27
Solid = -
Blocks = 1

  0%    
Would you like to replace the existing file:
  Path:     ./orders.xml
  Size:     371359286 bytes (355 MiB)
  Modified: 2013-10-23 13:02:38
with the file from archive:
  Path:     orders.xml
  Size:     371359286 bytes (355 MiB)
  Modified: 2013-10-23 13:02:38
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? Y

  0% - orders.xml                   7% - orders.xml                  15% - order

Used heappq module to implement Priority Queue in Python.
Used threading to perform simultaneous execution.

Buy works as Delete From Heap using heap pop. $O(log N)$ time.

Sell works as Push in to Heap using heap push. $O(log N)$ time.

In [12]:
import xml.etree.ElementTree as ET

from heapq import *
from time import *
from datetime import datetime
import threading
from tabulate import tabulate

tree = ET.parse('orders.xml')
root = tree.getroot()
order_len = len(root)

buy = []
sell = []
dele = []

def printTable():
    for i in range(3):
        print("book :  Book-{}".format(i+1))
        head = ["BUY", "SELL"]
        mydata = []
        buyCount = 0
        m = len(buy[i])
        for j in range(m):
            [a, c, v] = heappop(buy[i])
            if (str(c) in dele[i]):
                continue
            else:
                p = ("{} @ {}".format(v, (-1) * a))
                mydata.append([p, ""])
                buyCount += 1

        sellCount = 0
        n = len(sell[i])
        for j in range(n):
            [a, c, v] = heappop(sell[i])
            if (str(c) in dele[i]):
                continue
            else:
                p = ("{} @ {}".format(v, a))
                if(sellCount < buyCount):
                    mydata[sellCount][1] = p
                    sellCount += 1
                else:
                    mydata.append(["", p])

        print(tabulate(mydata, headers=head, tablefmt="grid"))
    print("Done")
    
def AddOrder(boo, op, price, vol, id):
    global buy, sell, buy2, sell2, dele
    n = int(boo[5:]) - 1
    while(len(buy) < n+1):
        p = []
        q = []
        d = set()
        heapify(p)
        heapify(q)
        buy.append(p)
        sell.append(q)
        dele.append(d)

    if(op == "SELL"):
        m = len(buy[n])
        if(m == 0):
            heappush(sell[n], [price, id, vol])
            return

        [a, c, v] = heappop(buy[n])
        while(str(c) in dele[n]):
            [a, c, v] = heappop(buy[n])

        if(price > (-1)*a):
            heappush(buy[n], [a, c, v])
            heappush(sell[n], [price, id, vol])

        else:
            if(v == vol):
                return
            elif(v > vol):
                heappush(buy[n], [a, c, v-vol])
            else:
                AddOrder(boo, op, price, vol - v, id)

    else:
        m = len(sell[n])
        if(m == 0):
            heappush(buy[n], [(-1)*price, id, vol])
            return

        [a, c, v] = heappop(sell[n])
        while (str(c) in dele[n]):
            [a, c, v] = heappop(sell[n])

        if(price < a):
            heappush(sell[n], [a, c, v])
            heappush(buy[n], [(-1)*price, id, vol])

        else:
            if(v == vol):
                return
            elif(v > vol):
                heappush(sell[n], [a, c, v-vol])
            else:
                AddOrder(boo, op, price, vol - v, id)

def DeleteOrder(b, id):
    global dele
    n = int(b[5:]) - 1
    dele[n].add(id)


print("Time Stamp (Process Started): {}".format(datetime.today()))
st = time()
def fetch(threadNo):
    for i in range(order_len):
        a = root[i].attrib
        alen = len(a)
        y = int(a['book'][5:])
        if(y != threadNo):
            continue
        elif(alen == 2):
            DeleteOrder(a['book'], a['orderId'])
        else:
            AddOrder(a['book'], a['operation'], float(a['price']), int(a['volume']), int(a['orderId']))


threads = []
startTime = time()
noOfThreads = 3

for i in range(noOfThreads):
    x = threading.Thread(target=fetch, args=(i+1,))
    threads.append(x)
    x.start()

for i in range(noOfThreads):
    threads[i].join()

printTable()
print("Time Stamp (Process Ended): {}".format(datetime.today()))
et = time()
print("Execution Time: {} seconds".format(et - st))


Time Stamp (Process Started): 2022-12-06 17:34:25.370027
book :  Book-1
+-----------+------------+
| BUY       | SELL       |
+===========+============+
| 89 @ 99.9 | 13 @ 100.3 |
+-----------+------------+
| 3 @ 99.9  | 84 @ 100.3 |
+-----------+------------+
| 24 @ 99.9 | 62 @ 100.4 |
+-----------+------------+
| 50 @ 99.8 | 45 @ 100.4 |
+-----------+------------+
| 40 @ 99.7 | 53 @ 100.5 |
+-----------+------------+
| 21 @ 99.7 | 88 @ 100.5 |
+-----------+------------+
| 71 @ 99.6 | 84 @ 100.5 |
+-----------+------------+
| 85 @ 99.6 | 85 @ 100.5 |
+-----------+------------+
| 50 @ 99.6 | 76 @ 100.5 |
+-----------+------------+
| 67 @ 99.6 | 20 @ 100.6 |
+-----------+------------+
| 69 @ 99.6 | 34 @ 100.6 |
+-----------+------------+
| 85 @ 99.5 | 55 @ 100.7 |
+-----------+------------+
| 50 @ 99.5 | 63 @ 100.8 |
+-----------+------------+
| 1 @ 99.5  | 38 @ 100.9 |
+-----------+------------+
| 2 @ 99.4  | 69 @ 100.9 |
+-----------+------------+
| 66 @ 99.4 | 35 @ 100.9 |
+---------